In [1]:
from ultralytics import YOLO
import cv2
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image
import matplotlib.pyplot as plt
from gtts import gTTS
import numpy as np
import IPython.display as display
import translators as ts


# Leer nuestro modelo
model_yolo = YOLO("best.pt")

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [2]:
def describe_person(image):
    description = []
    
    #text = "Is a man or a woman?"
    #encoding = processor(image, text, return_tensors="pt")
    #outputs = model(**encoding)
    #logits = outputs.logits
    #idx = logits.argmax(-1).item()
    #description.append("A " + model.config.id2label[idx])
    
    text = "What is the color of  the pants?"
    encoding = processor(image, text, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    description.append("A person with " + model.config.id2label[idx] + " pants, ")
    
    text = "What is the color of  the shirt ?"
    encoding = processor(image, text, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    description.append("and " + model.config.id2label[idx] + " shirt" )
    
    #text = "What color is the jacket?"
    #encoding = processor(image, text, return_tensors="pt")
    #outputs = model(**encoding)
    #logits = outputs.logits
    #idx = logits.argmax(-1).item()
    #description.append(model.config.id2label[idx] + " jacket")
    
    return description

In [3]:
#Realizar VideoCaptura
#cap = cv2.VideoCapture(0)
#frame = cv2.imread("/kaggle/input/data-tst/ima 1.png")


# Realizar VideoCaptura
cap = cv2.VideoCapture(0)
corte = 0

# Bucle
while True:

    ret, frame = cap.read()
    
    
    # Leemos resultados
    resultados = model_yolo.predict(frame, imgsz=640, conf=0.60, verbose=False)    
        
    boxes = resultados[0].boxes.xyxy.tolist()
    classes = resultados[0].boxes.cls.tolist()
    names = resultados[0].names
    confidences = resultados[0].boxes.conf.tolist()
        
    #print(boxes , classes , names , confidences)
    
    # Mostramos resultados
    anotaciones = resultados[0].plot()
    
    try:
        indice = classes.index(1)        
        crop_img=frame[int(boxes[indice][1]): int(boxes[indice][3]), int(boxes[indice][0]) : int(boxes[indice][2])]        
    except:
        pass    
    
    #Etiqueta
    if 1 in classes:  
        cv2.imshow("DETECCION Y SEGMENTACION", crop_img)
        
        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        descripcion = describe_person(crop_img)
        cadena = "".join(descripcion)
        cadena = ts.translate_text(cadena, translator='google', to_language='es')
        language = 'es'
        myobj = gTTS(text=cadena, lang=language, slow=False)   
        myobj.save("welcome.mp3")          
        print(cadena) 
             
        
        #corte = 1
    try:
        crop_img_color = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        cv2.imshow("ImgCut", crop_img_color)
    except:
        pass
    cv2.imshow("DETECCION Y SEGMENTACION", anotaciones)
    # Cerrar nuestro programa
    if cv2.waitKey(1) == 27 or corte == 1:
        break
    
cap.release()
cv2.destroyAllWindows()    #display.Audio("/kaggle/working/welcome.mp3", autoplay=True)# audio 
display.Audio("welcome.mp3", autoplay=True)

Una persona con pantalones negros y camisa naranja


In [ ]:
# Realizar VideoCaptura
cap = cv2.VideoCapture(0)

# Bucle
while True:
    # Leer nuestros fotogramas
    ret, frame = cap.read()

    # Leemos resultados
    resultados = model_yolo.predict(frame, imgsz=640, conf=0.60)

    # Mostramos resultados
    anotaciones = resultados[0].plot()

    # Mostramos nuestros fotogramas
    cv2.imshow("DETECCION Y SEGMENTACION", anotaciones)

    # Cerrar nuestro programa
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
display.Audio("/kaggle/working/welcome.mp3", autoplay=True)

In [78]:
if 0 in classes: # Imprime lo de abajo
    print("El número 0 existe en la lista")
    
indice = classes.index(0)
print("El número 50 está en la posición {} de la lista".format(indice))

El número 0 existe en la lista
El número 50 está en la posición 0 de la lista


In [16]:
classes = resultados[0].boxes.cls.tolist()
print(classes)

[]


In [31]:
# Bucle
while True:
    
    crop_img=anotaciones[3:480, 98 : 499]
    
    # Mostramos nuestros fotogramas
    cv2.imshow("DETECCION Y SEGMENTACION", anotaciones)
    
    cv2.imshow("Persona", crop_img )

    # Cerrar nuestro programa
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [23]:
boxes[0][2]

499.49896240234375

In [30]:
# prepare image + question
url = "https://img.freepik.com/foto-gratis/foto-joven-atractivo-modelo-pie-levantando-mano-foto-alta-calidad_144627-60066.jpg"
#image = Image.open("/kaggle/input/data-tst/WhatsApp Image 2024-04-19 at 13.12.20_52ef439e.jpg")
#image = Image.open(requests.get(url, stream=True).raw)
text = "Is he with the hand rise?"

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# prepare inputs
encoding = processor(crop_img, text, return_tensors="pt")

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

Predicted answer: no


In [4]:
# Import the required module for text 
# to speech conversion
from gtts import gTTS

# This module is imported so that we can 
# play the converted audio
import os

# The text that you want to convert to audio
mytext = 'Welcome to geeksforgeeks!'

# Language in which you want to convert
language = 'en'

# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)

# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")



sh: 1: start: not found


32512

In [6]:
import IPython.display as display

# audio
display.Audio("/kaggle/working/welcome.mp3", autoplay=True)
